In [10]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", 2) \
    .config("spark.driver.cores", 2) \
    .appName("Learning DataFrames") \
    .getOrCreate()                                 

In [15]:
data = [('Max', 55),
        ('Yan', 53),
        ('Dmitry', 54),
        ('Ann', 25)
        ]

columns = ['Name', 'Age']
df = spark.createDataFrame(data=data, schema=columns) 

In [16]:
df.show()

+------+---+
|  Name|Age|
+------+---+
|   Max| 55|
|   Yan| 53|
|Dmitry| 54|
|   Ann| 25|
+------+---+


In [17]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)


In [13]:
spark.stop()

In [1]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

import datetime

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
spark = SparkSession \
    .builder \
    .master("yarn") \
    .appName("alexpa_4_2") \
    .config("spark.ui.port", "4050") \
    .config("spark.yarn.queue", "root") \
    .getOrCreate()

spark = SparkSession.builder.getOrCreate()
print(datetime.datetime.now())

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2024-02-19 13:05:51,350 WARN util.Utils: Your hostname, fhmv3b8qjqqcv8g34hsf resolves to a loopback address: 127.0.1.1; using 172.16.0.22 instead (on interface eth0)
2024-02-19 13:05:51,351 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-02-19 13:05:53,273 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update 

2024-02-19 13:06:24.087120


In [2]:
events = spark.read.load(path = "/user/master/data/events/date=2022-05-25", format = 'json') 
events.show(10)

2024-02-19 13:06:47,759 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2024-02-19 13:07:02,759 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2024-02-19 13:07:17,758 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2024-02-19 13:07:32,758 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2024-02-19 13:07:47,759 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2024-02-19 13:08:02,758 WARN cluster.YarnScheduler: Initial job has no

+--------------------+------------+
|               event|  event_type|
+--------------------+------------+
|[,, 2022-05-25 17...|subscription|
|[,, 2022-05-25 15...|subscription|
|[,, 2022-05-25 18...|subscription|
|[,, 2022-05-25 00...|subscription|
|[,, 2022-05-25 03...|subscription|
|[,, 2022-05-25 04...|subscription|
|[,, 2022-05-25 15...|subscription|
|[,, 2022-05-25 01...|subscription|
|[,, 2022-05-25 00...|subscription|
|[,, 2022-05-25 07...|subscription|
+--------------------+------------+
only showing top 10 rows



In [5]:
!hdfs dfs -ls /user/master/data/snapshots/channels/actual

Found 2 items
-rw-r--r--   5 hdfs hadoop          0 2023-09-25 20:34 /user/master/data/snapshots/channels/actual/_SUCCESS
-rw-r--r--   5 hdfs hadoop    4295876 2023-09-25 20:34 /user/master/data/snapshots/channels/actual/part-00000-beae86b3-af54-4415-8f29-31dd79cfe178-c000.snappy.parquet


In [6]:
df = spark.read.parquet("/user/master/data/snapshots/channels/actual/part-00000-beae86b3-af54-4415-8f29-31dd79cfe178-c000.snappy.parquet")

In [7]:
df.write.option("header",True) \
        .partitionBy("channel_type") \
        .mode("append") \
        .parquet("/user/timefor/analytics/test") 

In [9]:
!hdfs dfs -ls /user/timefor/analytics/test

Found 3 items
-rw-r--r--   1 timefor timefor          0 2024-02-19 13:41 /user/timefor/analytics/test/_SUCCESS
drwxr-xr-x   - timefor timefor          0 2024-02-19 13:41 /user/timefor/analytics/test/channel_type=channel
drwxr-xr-x   - timefor timefor          0 2024-02-19 13:41 /user/timefor/analytics/test/channel_type=river


In [13]:
df1 = spark.read.parquet("/user/timefor/analytics/test").select("channel_type").orderBy("channel_type").distinct().show()

+------------+
|channel_type|
+------------+
|       river|
|     channel|
+------------+



In [14]:
spark.stop()

In [15]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Learning DataFrames") \
                    .getOrCreate()
# данные первого датафрейма 
book = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
        ('Nineteen Eighty-Four', 'George Orwell', 382),
        ('Jane Eyre', 'Charlotte Brontë', 159),
        ('Catch-22', 'Joseph Heller',  174),
        ('The Catcher in the Rye', 'J. D. Salinger',  168),
        ('The Wind in the Willows', 'Kenneth Grahame',  259),
        ('The Mayor of Casterbridge', 'Thomas Hardy',  300),
        ('Bad Girls', 'Jacqueline Wilson',  299)
]
# данные второго датафрейма
library = [
        ( 322, "1"),
        ( 250, "2" ),
        (400, "2"),
        (159, "1"),
        (382, "2"),
        (322, "1")
]
# названия атрибутов
columns = ['title', 'author', 'book_id']
columns_library = ['book_id', 'Library_id']
# создаём датафреймы
df = spark.createDataFrame(data=book, schema=columns)
df_library  = spark.createDataFrame(data=library, schema=columns_library )


In [17]:
df.show()

+--------------------+-----------------+-------+
|               title|           author|book_id|
+--------------------+-----------------+-------+
|Harry Potter and ...|    J. K. Rowling|    322|
|Nineteen Eighty-Four|    George Orwell|    382|
|           Jane Eyre| Charlotte Brontë|    159|
|            Catch-22|    Joseph Heller|    174|
|The Catcher in th...|   J. D. Salinger|    168|
|The Wind in the W...|  Kenneth Grahame|    259|
|The Mayor of Cast...|     Thomas Hardy|    300|
|           Bad Girls|Jacqueline Wilson|    299|
+--------------------+-----------------+-------+


In [18]:
joined = df.join(df_library, df.book_id == df_library.book_id, how='leftanti').drop(df.book_id )

In [27]:
joined.select('title').distinct().show(truncate=False)

+-------------------------+
|title                    |
+-------------------------+
|The Mayor of Casterbridge|
|Catch-22                 |
|The Catcher in the Rye   |
|The Wind in the Willows  |
|Bad Girls                |
+-------------------------+


In [28]:
joined.select('title').distinct().count()

5

In [29]:
df.join(df_library, df.book_id == df_library.book_id, how='left') \
    .drop(df.book_id ) \
    .select('title') \
    .show(truncate=False)

+-----------------------------------+
|title                              |
+-----------------------------------+
|Bad Girls                          |
|Catch-22                           |
|The Catcher in the Rye             |
|Nineteen Eighty-Four               |
|Harry Potter and the Goblet of Fire|
|Harry Potter and the Goblet of Fire|
|Jane Eyre                          |
|The Mayor of Casterbridge          |
|The Wind in the Willows            |
+-----------------------------------+


In [30]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Learning DataFrames") \
    .getOrCreate()
# данные первого датафрейма 
book_1 = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
          ('Nineteen Eighty-Four', 'George Orwell', 382),
          ('Jane Eyre', 'Charlotte Brontë', 159),
          ('Catch-22', 'Joseph Heller',  174),
          ('The Catcher in the Rye', 'J. D. Salinger',  168),
          ('The Wind in the Willows', 'Kenneth Grahame',  259),
          ('The Mayor of Casterbridge', 'Thomas Hardy',  300),
          ('Bad Girls', 'Jacqueline Wilson',  299)
          ]
# данные второго датафрейма
book_2 = [
    ('Black Beauty',657 ,'Anna Sewell'),
    ('Artemis Fowl',558,'Eoin Colfer'),
    ('The Magic Faraway Tree', 567,'Enid Blyton'),
    ('The Witches', 567,'Roald Dahl'),
    ('Frankenstein',567 ,'Mary Shelley'),
    ('The Little Prince',557 ,'Antoine de Saint-Exupéry'),
    ('The Truth', 576 ,'Terry Pratchett')
]
# названия атрибутов
columns_1= ['title', 'author', 'book_id']
columns_2 = ['title', 'book_id', 'author']
# создаём датафреймы
df_1 = spark.createDataFrame(data=book_1 , schema=columns_1)
df_2  = spark.createDataFrame(data=book_2 , schema=columns_2)
# напишите ваш код ниже


In [32]:
df_1.unionByName(df_2).show()

+--------------------+--------------------+-------+
|               title|              author|book_id|
+--------------------+--------------------+-------+
|Harry Potter and ...|       J. K. Rowling|    322|
|Nineteen Eighty-Four|       George Orwell|    382|
|           Jane Eyre|    Charlotte Brontë|    159|
|            Catch-22|       Joseph Heller|    174|
|The Catcher in th...|      J. D. Salinger|    168|
|The Wind in the W...|     Kenneth Grahame|    259|
|The Mayor of Cast...|        Thomas Hardy|    300|
|           Bad Girls|   Jacqueline Wilson|    299|
|        Black Beauty|         Anna Sewell|    657|
|        Artemis Fowl|         Eoin Colfer|    558|
|The Magic Faraway...|         Enid Blyton|    567|
|         The Witches|          Roald Dahl|    567|
|        Frankenstein|        Mary Shelley|    567|
|   The Little Prince|Antoine de Saint-...|    557|
|           The Truth|     Terry Pratchett|    576|
+--------------------+--------------------+-------+


In [35]:
joined.cache()

DataFrame[title: string, author: string]

In [36]:
joined.explain()

== Physical Plan ==
InMemoryTableScan [title#66, author#67]
   +- InMemoryRelation [title#66, author#67], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(5) Project [title#66, author#67]
            +- SortMergeJoin [book_id#68L], [book_id#72L], LeftAnti
               :- *(2) Sort [book_id#68L ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(book_id#68L, 200), true, [id=#640]
               :     +- *(1) Scan ExistingRDD[title#66,author#67,book_id#68L]
               +- *(4) Sort [book_id#72L ASC NULLS FIRST], false, 0
                  +- Exchange hashpartitioning(book_id#72L, 200), true, [id=#645]
                     +- *(3) Project [book_id#72L]
                        +- *(3) Filter isnotnull(book_id#72L)
                           +- *(3) Scan ExistingRDD[book_id#72L,Library_id#73]


In [39]:
spark.sparkContext.setCheckpointDir("/user/timefor/analytics/test_check")
joined.checkpoint()

DataFrame[title: string, author: string]

In [40]:
joined.explain()

== Physical Plan ==
InMemoryTableScan [title#66, author#67]
   +- InMemoryRelation [title#66, author#67], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(5) Project [title#66, author#67]
            +- SortMergeJoin [book_id#68L], [book_id#72L], LeftAnti
               :- *(2) Sort [book_id#68L ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(book_id#68L, 200), true, [id=#640]
               :     +- *(1) Scan ExistingRDD[title#66,author#67,book_id#68L]
               +- *(4) Sort [book_id#72L ASC NULLS FIRST], false, 0
                  +- Exchange hashpartitioning(book_id#72L, 200), true, [id=#645]
                     +- *(3) Project [book_id#72L]
                        +- *(3) Filter isnotnull(book_id#72L)
                           +- *(3) Scan ExistingRDD[book_id#72L,Library_id#73]


In [41]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Learning DataFrames") \
    .getOrCreate()
# данные первого датафрейма

book = [('Harry Potter and the Goblet of Fire', 'J. K. Rowling', 322),
        ('Nineteen Eighty-Four', 'George Orwell', 382),
        ('Jane Eyre', 'Charlotte Brontë', 159),
        ('Catch-22', 'Joseph Heller',  174),
        ('The Catcher in the Rye', 'J. D. Salinger',  168),
        ('The Wind in the Willows', 'Kenneth Grahame',  259),
        ('The Mayor of Casterbridge', 'Thomas Hardy',  300),
        ('Bad Girls', 'Jacqueline Wilson',  299)
        ]

# данные второго датафрейма
library = [
    ( 322, "1"),
    ( 250, "2" ),
    (400, "2"),
    (159, "1"),
    (382, "2"),
    (322, "1")
]

# названия атрибутов
columns = ['title', 'author', 'book_id']
columns_library = ['book_id', 'Library_id']

# создаём датафреймы
df = spark.createDataFrame(data=book, schema=columns)
df_library  = spark.createDataFrame(data=library, schema=columns_library )

# делаем join
df_join = df.join(df_library,['book_id'], 'leftanti').select('title')
df_cache= df_join.cache()

spark.sparkContext.setCheckpointDir("/user/timefor/analytics/test_check")
df_checkpoint=df_join.checkpoint()

In [42]:
df_join.explain()

== Physical Plan ==
InMemoryTableScan [title#264]
   +- InMemoryRelation [title#264], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(5) Project [title#264]
            +- SortMergeJoin [book_id#266L], [book_id#270L], LeftAnti
               :- *(2) Sort [book_id#266L ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(book_id#266L, 200), true, [id=#691]
               :     +- *(1) Project [title#264, book_id#266L]
               :        +- *(1) Scan ExistingRDD[title#264,author#265,book_id#266L]
               +- *(4) Sort [book_id#270L ASC NULLS FIRST], false, 0
                  +- Exchange hashpartitioning(book_id#270L, 200), true, [id=#696]
                     +- *(3) Project [book_id#270L]
                        +- *(3) Filter isnotnull(book_id#270L)
                           +- *(3) Scan ExistingRDD[book_id#270L,Library_id#271]


In [43]:
spark.stop()

In [44]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Learning DataFrames") \
    .getOrCreate()
events = spark.read.json("/user/master/data/events/")
events.show(10) 

+--------------------+------------+----------+
|               event|  event_type|      date|
+--------------------+------------+----------+
|[,, 2022-05-25 17...|subscription|2022-05-25|
|[,, 2022-05-25 15...|subscription|2022-05-25|
|[,, 2022-05-25 18...|subscription|2022-05-25|
|[,, 2022-05-25 00...|subscription|2022-05-25|
|[,, 2022-05-25 03...|subscription|2022-05-25|
|[,, 2022-05-25 04...|subscription|2022-05-25|
|[,, 2022-05-25 15...|subscription|2022-05-25|
|[,, 2022-05-25 01...|subscription|2022-05-25|
|[,, 2022-05-25 00...|subscription|2022-05-25|
|[,, 2022-05-25 07...|subscription|2022-05-25|
+--------------------+------------+----------+


In [47]:
import pyspark.sql.functions as F
events_curr_day = events.withColumn('current_date',F.current_date())
events_curr_day.show(10, False)

+---------------------------------------------------+------------+----------+------------+
|event                                              |event_type  |date      |current_date|
+---------------------------------------------------+------------+----------+------------+
|[,, 2022-05-25 17:04:31,,,,,,,,,,, 913681,,, 48430]|subscription|2022-05-25|2024-02-19  |
|[,, 2022-05-25 15:20:12,,,,,,,,,,, 579406,,, 84256]|subscription|2022-05-25|2024-02-19  |
|[,, 2022-05-25 18:49:53,,,,,,,,,,, 84145,,, 48457] |subscription|2022-05-25|2024-02-19  |
|[,, 2022-05-25 00:15:15,,,,,,,,,,, 344598,,, 84308]|subscription|2022-05-25|2024-02-19  |
|[,, 2022-05-25 03:20:22,,,,,,,,,,, 596627,,, 48474]|subscription|2022-05-25|2024-02-19  |
|[,, 2022-05-25 04:54:08,,,,,,,,,,, 767711,,, 62234]|subscription|2022-05-25|2024-02-19  |
|[,, 2022-05-25 15:39:11,,,,,,,,,,, 657579,,, 69692]|subscription|2022-05-25|2024-02-19  |
|[,, 2022-05-25 01:12:32,,,,,,,,,,, 822105,,, 62257]|subscription|2022-05-25|2024-02-19  |

In [48]:
events_diff = events_curr_day \
    .withColumn('date', F.lit('2022-05-31')) \
    .withColumn('diff',F.datediff(F.col('current_date'),F.col('date'))) \
    .show()

+--------------------+------------+----------+------------+----+
|               event|  event_type|      date|current_date|diff|
+--------------------+------------+----------+------------+----+
|[,, 2022-05-25 17...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 15...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 18...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 00...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 03...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 04...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 15...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 01...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 00...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 07...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 13...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 22...|subscription|2022-05-31|  2024-02-19| 629|
|[,, 2022-05-25 11...|sub

In [49]:
events = spark.read.json("/user/master/data/events/date=2022-05-31")

In [64]:
events.show(truncate=False)

+-------------------------------------------------+------------+
|event                                            |event_type  |
+-------------------------------------------------+------------+
|[,, 2022-05-31 06:03:31,,,,,,,,, 966719,, 100029]|subscription|
|[,, 2022-05-31 07:14:34,,,,,,,,, 636643,, 32034] |subscription|
|[,, 2022-05-31 00:42:22,,,,,,,,, 724091,, 83285] |subscription|
|[,, 2022-05-31 16:58:26,,,,,,,,, 294429,, 54960] |subscription|
|[,, 2022-05-31 00:57:03,,,,,,,,, 289857,, 8335]  |subscription|
|[,, 2022-05-31 04:44:44,,,,,,,,, 246671,, 54975] |subscription|
|[,, 2022-05-31 01:59:08,,,,,,,,, 708394,, 100411]|subscription|
|[,, 2022-05-31 02:07:53,,,,,,,,, 24230,, 32085]  |subscription|
|[,, 2022-05-31 06:23:12,,,,,,,,, 28869,, 83526]  |subscription|
|[,, 2022-05-31 15:26:41,,,,,,,,, 294429,, 5523]  |subscription|
|[,, 2022-05-31 00:00:02,,,,,,,,, 433766,, 100621]|subscription|
|[,, 2022-05-31 10:45:23,,,,,,,,, 884358,, 32389] |subscription|
|[,, 2022-05-31 03:06:37,

In [67]:
events_with_hms = events \
    .withColumn('hour', F.hour(F.col('event.datetime'))) \
    .withColumn('minute', F.minute(F.col('event.datetime'))) \
    .withColumn('second', F.second(F.col('event.datetime'))) \
    .sort(F.col('event.datetime').desc())


In [70]:
events_with_hms.show(10)

+--------------------+------------+----+------+------+
|               event|  event_type|hour|minute|second|
+--------------------+------------+----+------+------+
|[[19342], 987160,...|     message|  23|    57|    38|
|[,, 2022-05-31 23...|subscription|  23|    56|    58|
|[[26358], 247511,...|     message|  23|    56|    53|
|[[79792], 748847,...|     message|  23|    56|    27|
|[,, 2022-05-31 23...|subscription|  23|    53|    42|
|[,, 2022-05-31 23...|subscription|  23|    53|    14|
|[[151897], 396845...|     message|  23|    52|    15|
|[,, 2022-05-31 23...|subscription|  23|    52|     8|
|[,, 2022-05-31 23...|subscription|  23|    51|    19|
|[,, 2022-05-31 23...|subscription|  23|    51|    15|
+--------------------+------------+----+------+------+


In [71]:
events.filter(F.col('event.message_from').isNull()).count()

227667

In [72]:
events.filter(F.col('event.message_to').isNotNull()).count()

221

In [73]:
events.count() - events.na.drop(subset='event.message_from').count()

227667

In [74]:
events.filter(F.col('event_type')=='message').groupBy(F.col('event.message_from')).count().show()

+------------+-----+
|message_from|count|
+------------+-----+
|       77947|    1|
|      142511|    1|
|      153557|    1|
|       99185|    1|
|      160744|    1|
|       24880|    2|
|       51165|    1|
|       38941|    1|
|       51349|    1|
|       51534|    1|
|      126842|    1|
|      159521|    1|
|       91431|    1|
|       87917|    1|
|       27813|    1|
|       33851|    1|
|      166941|    1|
|      100323|    1|
|       22052|    1|
|       75949|    1|
+------------+-----+


In [75]:
events_1 = spark.read.json("/user/master/data/events/date=2022-05-25")

In [79]:
max_reactions = events_1 \
    .filter(F.col('event_type') == 'reaction') \
    .groupBy(F.col('event.reaction_from')) \
    .count() \
    .agg(F.max('count').alias('max_reactions'))

max_reactions.show()

+-------------+
|max_reactions|
+-------------+
|           11|
+-------------+


9. Оконные функции

In [2]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [5]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Learning DataFrames") \
    .getOrCreate()
# данные  датафрейма 
data = [('2021-01-06', 3744, 63, 322),
        ('2021-01-04', 2434, 21, 382),
        ('2021-01-04', 2434, 32, 159),
        ('2021-01-05', 3744, 32, 159),
        ('2021-01-06', 4342, 32, 159),
        ('2021-01-05', 4342, 12, 259),
        ('2021-01-06', 5677, 12, 259),
        ('2021-01-04', 5677, 23, 499)
        ]
# названия атрибутов
columns = ['dt', 'user_id', 'product_id', 'purchase_amount']
# создаём датафрейм
df = spark.createDataFrame(data=data, schema=columns) 

In [7]:
# импортируем оконную функцию и модуль Spark Functions
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# создаём объект оконной функции
window = Window().partitionBy(['dt']).orderBy(F.asc('purchase_amount'))

# создаём колонку с рассчитанной статистикой по оконной функции
df_window = df.withColumn("rank", F.row_number().over(window))

# выводим нужные колонки
df_window.select('dt', 'user_id', 'rank', 'purchase_amount').show()

+----------+-------+----+---------------+
|        dt|user_id|rank|purchase_amount|
+----------+-------+----+---------------+
|2021-01-04|   2434|   1|            159|
|2021-01-04|   2434|   2|            382|
|2021-01-04|   5677|   3|            499|
|2021-01-05|   3744|   1|            159|
|2021-01-05|   4342|   2|            259|
|2021-01-06|   4342|   1|            159|
|2021-01-06|   5677|   2|            259|
|2021-01-06|   3744|   3|            322|
+----------+-------+----+---------------+


In [6]:
# создаем объект оконной функции
window = Window.orderBy(F.asc('purchase_amount'))

# создаем колонку с рассчитанной статистикой по оконной функции
df_window = df.withColumn("row_number", F.row_number().over(window))

# выводим нужные колонки
df_window.select('dt', 'user_id', 'purchase_amount', 'row_number').show()

NameError: name 'Window' is not defined

In [16]:
# создаём объект оконной функции
window = Window().partitionBy('user_id').orderBy('dt')

# создаём колонку с рассчитанной статистикой по оконной функции
dfWithLag = df.withColumn("lag_1",F.lag("purchase_amount", 1).over(window))

# фильтруем данные
dfWithLag .select('dt','user_id', 'purchase_amount','lag_1').show() 

+----------+-------+---------------+-----+
|        dt|user_id|purchase_amount|lag_1|
+----------+-------+---------------+-----+
|2021-01-04|   2434|            382| NULL|
|2021-01-04|   2434|            159|  382|
|2021-01-05|   3744|            159| NULL|
|2021-01-06|   3744|            322|  159|
|2021-01-05|   4342|            259| NULL|
|2021-01-06|   4342|            159|  259|
|2021-01-04|   5677|            499| NULL|
|2021-01-06|   5677|            259|  499|
+----------+-------+---------------+-----+


In [24]:
spark.stop()

In [1]:
import os
import pyspark.sql.functions as F
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
spark = SparkSession \
    .builder \
    .master("yarn") \
    .appName("alexpa_9_2") \
    .config("spark.ui.port", "4050") \
    .config("spark.yarn.queue", "root") \
    .getOrCreate()

spark = SparkSession.builder.getOrCreate()


24/02/20 17:41:28 WARN Utils: Your hostname, fhmbmgti0ul88u8innkd resolves to a loopback address: 127.0.1.1; using 172.16.0.39 instead (on interface eth0)
24/02/20 17:41:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 17:41:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/20 17:41:31 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/02/20 18:20:02 ERROR YarnClientSchedulerBackend: The YARN application has already ended! It might have been killed or the Application Master may have failed to start. Check the YARN application logs for more details.
24/02/20 18:20:02 ERROR SparkContext: Error initializing SparkContext.
org.apache.spark.SparkException: Application applicat

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Application application_1692104774102_32547 was killed by user dr.who
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.waitForApplication(YarnClientSchedulerBackend.scala:98)
	at org.apache.spark.scheduler.cluster.YarnClientSchedulerBackend.start(YarnClientSchedulerBackend.scala:65)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:235)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:604)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java

In [ ]:
events = spark.read.json("/user/master/data/events/date=2022-05-01")

In [8]:
window = Window().partitionBy('user_id')

df_window_agg= df.withColumn("avg",F.avg("purchase_amount").over(window)) \
    .withColumn("sum",F.sum("purchase_amount").over(window))
df_window_agg.show()

+----------+-------+----------+---------------+-----+---+
|        dt|user_id|product_id|purchase_amount|  avg|sum|
+----------+-------+----------+---------------+-----+---+
|2021-01-04|   2434|        21|            382|270.5|541|
|2021-01-04|   2434|        32|            159|270.5|541|
|2021-01-06|   3744|        63|            322|240.5|481|
|2021-01-05|   3744|        32|            159|240.5|481|
|2021-01-06|   4342|        32|            159|209.0|418|
|2021-01-05|   4342|        12|            259|209.0|418|
|2021-01-06|   5677|        12|            259|379.0|758|
|2021-01-04|   5677|        23|            499|379.0|758|
+----------+-------+----------+---------------+-----+---+


In [11]:
window = Window().partitionBy('user_id')

df_window_m= df.withColumn("max",F.max("purchase_amount").over(window)) \
    .withColumn("min",F.min("purchase_amount").over(window))
df_window_m.distinct().show()

+----------+-------+----------+---------------+---+---+
|        dt|user_id|product_id|purchase_amount|max|min|
+----------+-------+----------+---------------+---+---+
|2021-01-04|   2434|        21|            382|382|159|
|2021-01-04|   2434|        32|            159|382|159|
|2021-01-06|   3744|        63|            322|322|159|
|2021-01-05|   3744|        32|            159|322|159|
|2021-01-06|   4342|        32|            159|259|159|
|2021-01-05|   4342|        12|            259|259|159|
|2021-01-06|   5677|        12|            259|499|259|
|2021-01-04|   5677|        23|            499|499|259|
+----------+-------+----------+---------------+---+---+


In [12]:
# Группировка по user_id и агрегация для нахождения максимальной и минимальной суммы покупки
df_grouped = df.groupBy("user_id") \
    .agg(
    F.max("purchase_amount").alias("max"),
    F.min("purchase_amount").alias("min")
)

df_grouped.show()

+-------+---+---+
|user_id|max|min|
+-------+---+---+
|   2434|382|159|
|   4342|259|159|
|   5677|499|259|
|   3744|322|159|
+-------+---+---+


In [ ]:
spark.stop()